In [1]:
import pandas as pd
from itertools import islice

In [34]:
source = 'data/820Metrotech_test.txt'
clients = []
with open(source, 'r') as f:
    for i, client in enumerate(f):
        if 'Client' in client:
            client = client.replace(':',';',1)
            for j in range(i, i+3):
                n = f.next()
                if 'First' in n:
                    F = n
                if 'Last' in n:
                    L = n
                    
                j+= 1
               
            clients.append(client+';'+F+';'+L)
            #print n

In [35]:
df = pd.DataFrame(columns=['client', 'MAC','first','last'])

In [36]:
## Add Clients to dataframe
for client in clients:
    df = df.append(pd.Series(client.split(';'), index=['client','MAC','first','last']), ignore_index=True)

In [37]:
df.head(10)

,client,MAC,first,last
0,Client 1,MAC 00:00:00:00:00:00\n,First : Tue Apr 18 18:28:36 2017\n,Last : Tue Apr 18 19:48:04 2017\n
1,Client 1,MAC B0:70:2D:7E:C5:53\n,First : Tue Apr 18 18:42:15 2017\n,Last : Tue Apr 18 18:42:45 2017\n
2,Client 1,MAC 00:03:7F:00:00:00\n,First : Tue Apr 18 18:31:50 2017\n,Last : Tue Apr 18 18:31:50 2017\n
3,Client 1,MAC 00:04:20:F3:E1:10\n,First : Tue Apr 18 19:22:24 2017\n,Last : Tue Apr 18 19:22:24 2017\n
4,Client 1,MAC 00:04:4B:2F:66:A2\n,First : Tue Apr 18 18:31:24 2017\n,Last : Tue Apr 18 19:46:17 2017\n
5,Client 1,MAC 00:0A:E2:16:BC:2A\n,First : Tue Apr 18 18:28:47 2017\n,Last : Tue Apr 18 19:46:58 2017\n
6,Client 1,MAC 00:0A:F5:01:B9:D4\n,First : Tue Apr 18 19:11:47 2017\n,Last : Tue Apr 18 19:11:47 2017\n
7,Client 1,MAC 00:11:D9:15:73:6C\n,First : Tue Apr 18 19:10:49 2017\n,Last : Tue Apr 18 19:46:38 2017\n
8,Client 1,MAC 00:17:F2:96:6B:6C\n,First : Tue Apr 18 18:58:31 2017\n,Last : Tue Apr 18 18:58:31 2017\n
9,Client 1,MAC 00:19:9D:DE:83:C8\n,First : Tue Apr 18 19:23:53 2017\n,Last : Tue Apr 18 19:25:10 2017\n


In [38]:
## Clean up MACA Address column
df.MAC = df.MAC.str.replace('MAC ' , '')
df.MAC = df.MAC.str.replace(r'\n','')
df['first'] = df['first'].str.replace(' First      : ' , '')
df['first'] = df['first'].str.replace(r'\n','')
df['last'] = df['last'].str.replace(' Last       : ' , '')
df['last'] = df['last'].str.replace(r'\n','')

In [39]:
df.MAC.head()

0     00:00:00:00:00:00
1     B0:70:2D:7E:C5:53
2     00:03:7F:00:00:00
3     00:04:20:F3:E1:10
4     00:04:4B:2F:66:A2
Name: MAC, dtype: object

In [40]:
df['last'].head()

0     Tue Apr 18 19:48:04 2017
1     Tue Apr 18 18:42:45 2017
2     Tue Apr 18 18:31:50 2017
3     Tue Apr 18 19:22:24 2017
4     Tue Apr 18 19:46:17 2017
Name: last, dtype: object

In [41]:
num_first = df['first'].unique()
len(num_first)

1012

In [42]:
num_last = df['last'].unique()
# df['last'].value_counts()
len(num_last)

1033

In [43]:
## Get the number of Unique MAC addresses
num_macs = df.MAC.unique()
len(num_macs)

1162

In [44]:
## MAC addresses and the number of times they appear in the log
df.MAC.value_counts()

 78:4F:43:75:0E:7C    9
 58:BF:EA:BF:1E:40    8
 40:40:A7:93:77:64    4
 68:64:4B:D8:DA:42    4
 E8:2A:EA:46:21:C0    3
 A0:99:9B:04:52:8D    3
 70:10:5C:B1:E1:80    3
 F8:BB:BF:80:29:BC    3
 48:45:20:C4:42:D5    3
 30:E4:DB:39:47:2F    3
 90:C7:D8:8A:69:C8    3
 10:AE:60:6D:FF:7F    3
 DC:0C:5C:94:B2:E2    3
 A0:99:9B:41:65:AD    3
 2C:20:0B:B8:59:8F    3
 5C:AF:06:82:ED:F6    3
 2C:F0:A2:1E:D0:34    3
 00:00:5E:00:01:53    3
 64:9A:BE:E3:43:AE    3
 14:22:DB:26:B0:0E    3
 58:1F:AA:43:F0:9A    3
 14:22:DB:16:37:2E    3
 78:CA:39:B7:17:E8    2
 78:9F:70:A2:36:E9    2
 18:65:90:12:9A:57    2
 8C:2D:AA:35:E6:FB    2
 8C:2D:AA:35:A9:8F    2
 98:5F:D3:51:94:D8    2
 2C:33:61:9E:25:5C    2
 78:F8:82:B8:91:AA    2
                     ..
 9E:C6:3B:08:76:27    1
 60:6D:C7:C6:0E:2F    1
 DA:A1:19:B1:C7:B8    1
 B4:75:0E:80:B8:9C    1
 52:21:6C:45:A5:CF    1
 DA:A1:19:50:49:E1    1
 B6:F7:ED:D0:9B:AE    1
 DA:A1:19:EA:D3:66    1
 6A:69:8B:32:55:6F    1
 AE:9B:87:F9:9E:61    1
 A0:D7:95:05:9C: